In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import RNN
from keras.layers import GRU
from keras.layers import Dropout
from keras.layers import Activation
from keras.models import load_model
from keras.utils import plot_model

from statsmodels.tsa.arima.model import ARIMA

import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense, GRU
from tensorflow.keras.preprocessing import sequence

### Read data ###

In [5]:
df = pd.read_csv('./BTC-USD.csv')

In [6]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-03-01,3853.757080,3907.795410,3851.692383,3859.583740,3859.583740,7661247975
1,2019-03-02,3855.318115,3874.607422,3832.127930,3864.415039,3864.415039,7578786076
2,2019-03-03,3862.266113,3875.483643,3836.905762,3847.175781,3847.175781,7253558152
3,2019-03-04,3845.091553,3867.381836,3733.749756,3761.557129,3761.557129,9029175788
4,2019-03-05,3759.832520,3903.916748,3745.183105,3896.375000,3896.375000,10174126415


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1909 entries, 0 to 1908
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1909 non-null   object 
 1   Open       1909 non-null   float64
 2   High       1909 non-null   float64
 3   Low        1909 non-null   float64
 4   Close      1909 non-null   float64
 5   Adj Close  1909 non-null   float64
 6   Volume     1909 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 104.5+ KB


In [8]:
df.shape

(1909, 7)

### Preprocess data ###

In [9]:
df['Date'] = pd.to_datetime(df['Date'])